### 2. Binary Predict Nikkei Up or Downm
In 1. Predict Nikkei Up or Down, we tried to predict up and down, but loss or accuracy didn't improve.
I'm not sure why it didn't work.
So I'm following https://keras.io/getting-started/sequential-model-guide/ to make sure I have right setup of classification.

In [1]:
import sys
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils.visualize_util import plot
from matplotlib.pyplot import show, plot
import pandas as pd
import numpy as np
import math
import random
random.seed(0)


Using TensorFlow backend.


In [2]:
df = pd.read_csv("../../dont_remove_data/nikkei.csv", header=None)
df

,0,1,2,3,4,5
0,19960401,21600,21870,21600,21650,21840
1,19960402,21620,21720,21580,21650,14860
2,19960403,21750,21830,21410,21510,25180
3,19960404,21540,21670,21490,21670,15460
4,19960405,21640,21830,21610,21820,13940
5,19960408,21640,21660,21430,21480,14720
6,19960409,21660,21900,21640,21880,26280
7,19960410,21840,21950,21790,21840,16460
8,19960411,21730,21820,21670,21740,16110
9,19960412,21740,21880,21630,21780,23210


In [3]:
prices = df[1]
print(type(prices))
prices = prices.astype('float32')
mean = prices.mean()
print("mean", mean)
prices = prices / mean
prices[0]


<class 'pandas.core.series.Series'>
mean 13689.4


1.577863

In [4]:
# we use last 200 days data
nb_prev = 200

X = []
Y = []
# adjust Y to match the guidance
for i in range(len(prices) - nb_prev):
    X.append(np.array(prices.iloc[i:i + nb_prev]))
    up = (prices.iloc[i + nb_prev] - prices.iloc[i + nb_prev - 1]) > 0
    Y.append(1 if up else 0)
print("len(X)", len(X))
print("len(Y)", len(Y))
print("len(X[0])", len(X[0]))
print("np.count_nonzero(Y) =", np.count_nonzero(Y))

X = np.array(X).reshape((-1, nb_prev, 1))
Y = np.array(Y).reshape((-1, 1))

print("X.shape", X.shape)
print("Y.shape", Y.shape)


len(X) 4653
len(Y) 4653
len(X[0]) 200
np.count_nonzero(Y) = 2305
X.shape (4653, 200, 1)
Y.shape (4653, 1)


In [5]:
# Split train and test data
test_size = 0.1
nb_train = int(round(len(X) * (1 - test_size)))
print("test_data_size", len(X) - nb_train)

indices = np.random.permutation(X.shape[0])
training_idx, test_idx = indices[:nb_train], indices[nb_train:]
X_train, X_test = X[training_idx,:], X[test_idx,:]
Y_train, Y_test = Y[training_idx,:], Y[test_idx,:]

print(X_train.shape)
print(Y_train.shape)
print("X_test.shape", X_test.shape)
print("Y_test.shape", Y_test.shape)

test_data_size 465
(4188, 200, 1)
(4188, 1)
X_test.shape (465, 200, 1)
Y_test.shape (465, 1)


In [6]:
version = 1
in_out_neurons = 1
hidden_neurons = 30

model = Sequential()  
model.add(LSTM(hidden_neurons, batch_input_shape=(None, nb_prev, in_out_neurons), return_sequences=False))  
model.add(Dense(1))  
model.add(Activation("sigmoid"))

# https://keras.io/getting-started/sequential-model-guide/
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary)

<bound method Container.summary of <keras.models.Sequential object at 0x1113c1710>>


In [7]:
# Stacked LSTM for sequence classification
# from https://keras.io/getting-started/sequential-model-guide/
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(nb_prev, in_out_neurons)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print(model.summary)

<bound method Container.summary of <keras.models.Sequential object at 0x11264b5c0>>


### Train

In [8]:
# where we store optimized params
weights_dir = "{0}/Desktop/nikkei-up-down2-{1}".format(os.environ['HOME'], version)
print(weights_dir)

/Users/higepon/Desktop/nikkei-up-down2-1


In [ ]:
# train!
os.makedirs(weights_dir, exist_ok=True)
filepath = weights_dir + "/{loss:.4f}"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print(X_train.shape)
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, callbacks=callbacks_list)


(4188, 200, 1)
Epoch 1/10
4188/4188 [==============================] - 302s - loss: 8.0702 - acc: 0.4938   
Epoch 2/10
4188/4188 [==============================] - 188s - loss: 8.0702 - acc: 0.4938   
Epoch 3/10
4188/4188 [==============================] - 192s - loss: 8.0702 - acc: 0.4938   
Epoch 4/10
4188/4188 [==============================] - 195s - loss: 8.0702 - acc: 0.4938   
Epoch 5/10
4188/4188 [==============================] - 189s - loss: 8.0702 - acc: 0.4938   
Epoch 6/10
4188/4188 [==============================] - 189s - loss: 8.0702 - acc: 0.4938   
Epoch 7/10
4188/4188 [==============================] - 191s - loss: 8.0702 - acc: 0.4938   
Epoch 8/10
4188/4188 [==============================] - 245s - loss: 8.0702 - acc: 0.4938   
Epoch 9/10
2176/4188 [==============>...............] - ETA: 114s - loss: 8.1104 - acc: 0.4913

### Tentative conclusion
I tried various optimizer and loss functions, some are recommeneded in keras document.
But none of them worked. Accuracy is always arond 50%.

It kind of make sense, considering it's a hard problem.
I guess we need mroe signals to predict.

But to confirm my understanding, I'm getting back original nikkei fitting problem to see if it's aligned with this conclusion.
